In [ ]:
%pip install tensorflow
%pip install tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define data paths
train_dir = '/dataset/train'
val_dir = '/dataset/val'

# Define image size and batch size
img_height, img_width = 224, 224
batch_size = 32


In [ ]:
# Create data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
# Create data generator for validation
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
# Load EfficientNetB0 model (you can choose other versions like B3, B4, etc.)
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(img_height, img_width, 3)
)

In [ ]:

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Adjust the number of neurons as needed
predictions = Dense(train_generator.num_classes, activation='softmax')(x)


In [ ]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [ ]:
# Train the model
epochs = 10  # Adjust the number of epochs as needed
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=epochs
)

In [ ]:
# Unfreeze some layers for fine-tuning (optional)
for layer in base_model.layers[-20:]:
    layer.trainable = True


In [ ]:
# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy'])


In [ ]:

# Fine-tune the model
fine_tune_epochs = 5  # Adjust the number of epochs as needed
history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=fine_tune_epochs
)

In [ ]:
# Save the trained model
model.save('efficientnet_custom_dataset.h5')